# Sentence Similarity
We attempted to calculate the similarity between the review and the plot of the reviewed movie.

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


### Import Libraries

We used spaCy Sentence-BERT to calculate the similarity between the plot of the movie and the review.

In [1]:
import spacy_sentence_bert 
import pandas as pd 


In [4]:
nlp=spacy_sentence_bert.load_model('en_stsb_roberta_large')
similarityValue = []

/home/f.caprari/prova/Group10venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/f.caprari/prova/Group10venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Read the datasets

In [5]:
dataRew=pd.read_json("../Dataset/IMDB_reviews.json",lines=True)

In [16]:
dataMovie=pd.read_json('../Dataset/IMDB_movie_details.json',lines=True)

In [33]:
dataRewS=dataRew[["movie_id","review_text","is_spoiler"]]

In [34]:
dataRewS

,movie_id,review_text,is_spoiler
0,tt0111161,"In its Oscar year, Shawshank Redemption (writt...",True
1,tt0111161,The Shawshank Redemption is without a doubt on...,True
2,tt0111161,I believe that this film is the best story eve...,True
3,tt0111161,"**Yes, there are SPOILERS here**This film has ...",True
4,tt0111161,At the heart of this extraordinary movie is a ...,True
...,...,...,...
573908,tt0139239,"Go is wise, fast and pure entertainment. Assem...",False
573909,tt0139239,"Well, what shall I say. this one´s fun at any ...",False
573910,tt0139239,"Go is the best movie I have ever seen, and I'v...",False
573911,tt0139239,Call this 1999 teenage version of Pulp Fiction...,False


drop the film where the synopsis is not present

In [ ]:
dataMovie=dataMovie[dataMovie["plot_synopsis"]!='']

We use the end of the plot synopsis to calculate similarity because it is more likely to contain spoilers.

In [18]:
dataMovie['last'] = dataMovie['plot_synopsis'].apply(lambda x: x[-512:])

In [19]:
dataMovie

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis,last
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in...",realising that Ryan is leading them away from ...
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...,"arlier, and make a toast. Alas, the scotch has..."
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...,"to the city to sell the goods, but once it is ..."
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...,"Tracy, yet again, lying, cheating, seducing an..."
...,...,...,...,...,...,...,...,...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher...","scaring her away, he makes sure that she is ne..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ...",ies in the rain.Some time later Brandon is see...
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...,"the evacuation and, unable to handle the scorn..."
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,,


In [59]:
dataMovieS=dataMovie[["last","movie_id"]]

In [61]:
dataMovieS

,plot_synopsis,movie_id
0,"Jack Ryan (Ford) is on a ""working vacation"" in...",tt0105112
1,Four boys around the age of 10 are friends in ...,tt1204975
2,,tt0243655
3,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...,tt0040897
4,Jim McAllister (Matthew Broderick) is a much-a...,tt0126886
...,...,...
1567,"In the year 1998, Evan Treborn (Ashton Kutcher...",tt0289879
1568,"Brandon (Michael Fassbender) is a successful, ...",tt1723811
1569,The film alternates between three different pe...,tt5013056
1570,,tt0104014/


In [62]:
dataRewS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573913 entries, 0 to 573912
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   movie_id     573913 non-null  object
 1   review_text  573913 non-null  object
 2   is_spoiler   573913 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 9.3+ MB


### Merge  the two datasets based on movie_id

In [63]:
dataSimilar=dataRewS.merge(dataMovieS,left_on="movie_id",right_on="movie_id",how="left")

In [64]:
dataSimilar

,movie_id,review_text,is_spoiler,plot_synopsis
0,tt0111161,"In its Oscar year, Shawshank Redemption (writt...",True,"In 1947, Andy Dufresne (Tim Robbins), a banker..."
1,tt0111161,The Shawshank Redemption is without a doubt on...,True,"In 1947, Andy Dufresne (Tim Robbins), a banker..."
2,tt0111161,I believe that this film is the best story eve...,True,"In 1947, Andy Dufresne (Tim Robbins), a banker..."
3,tt0111161,"**Yes, there are SPOILERS here**This film has ...",True,"In 1947, Andy Dufresne (Tim Robbins), a banker..."
4,tt0111161,At the heart of this extraordinary movie is a ...,True,"In 1947, Andy Dufresne (Tim Robbins), a banker..."
...,...,...,...,...
573908,tt0139239,"Go is wise, fast and pure entertainment. Assem...",False,"The film is told out of chronological format, ..."
573909,tt0139239,"Well, what shall I say. this one´s fun at any ...",False,"The film is told out of chronological format, ..."
573910,tt0139239,"Go is the best movie I have ever seen, and I'v...",False,"The film is told out of chronological format, ..."
573911,tt0139239,Call this 1999 teenage version of Pulp Fiction...,False,"The film is told out of chronological format, ..."


In [ ]:
dataSimilar=dataRewS.dropna()

In [37]:
dataSimilar=dataSimilar.rename(columns={'last':'endfilm'})

### Try first on small dataset

In [65]:
dataReduced=dataSimilar[0:1000]

In [66]:
similarityValue=[]

### Compute the similarity

In [68]:
for i in range(dataReduced.count()[0]):
    QueryRew=nlp(dataReduced.loc[i]["review_text"])
    QueryFilm=nlp(dataReduced.loc[i]["endfilm"])
    Similarity=QueryRew.similarity(QueryFilm)
    tuples=(Similarity,dataReduced.loc[i]["is_spoiler"])
    similarityValue.append(tuples)
    print(Similarity,'|',dataReduced.loc[i]["is_spoiler"])

/tmp/ipykernel_1096221/2450476968.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in range(dataReduced.count()[0]):


0.33869002790979985 | True
0.46660456940234857 | True
0.5887434513481726 | True
0.3198871761876036 | True
0.5280436485594917 | True
0.501232109574961 | True
0.34240528211759347 | True
0.48278491864834805 | True
0.4765060588929023 | True
0.249723984480725 | True
0.6313566163235317 | True
0.26839212164268295 | True
0.4023244101006572 | True
0.25964126695267414 | True
0.599474619623344 | True
0.3411584473755826 | True
0.5033750131485597 | True
0.21751825390673138 | True
0.3657860382586075 | True
0.5568815968430768 | True
0.5245965601411626 | True
0.5074638952098678 | True
0.2765467850526387 | True
0.4166843516079689 | True
0.5495250188506189 | True
0.3728328310843555 | True
0.5740100300406288 | True
0.24977794559792837 | True
0.33867776063023175 | True
0.4256364965260205 | True
0.3016145338530643 | True
0.5665842707545304 | True
0.7386071644738266 | True
0.46766372857327576 | True
0.5012052296226946 | True
0.4611044731312414 | True
0.27568256511959177 | True
0.5379251326810418 | True
0.48

KeyboardInterrupt: 

### Note
We didn't apply the algorithm to the entire dataset because also testing on a small dataset, it becomes practically impossible to define a threshold value of similarity to determine whether a comment is a spoiler or not.